---

_You are currently looking at **version 1.1** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-data-analysis/resources/0dhYG) course resource._

---

In [418]:
import pandas as pd
import numpy as np
from scipy.stats import ttest_ind

# Assignment 4 - Hypothesis Testing
This assignment requires more individual learning than previous assignments - you are encouraged to check out the [pandas documentation](http://pandas.pydata.org/pandas-docs/stable/) to find functions or methods you might not have used yet, or ask questions on [Stack Overflow](http://stackoverflow.com/) and tag them as pandas and python related. And of course, the discussion forums are open for interaction with your peers and the course staff.

Definitions:
* A _quarter_ is a specific three month period, Q1 is January through March, Q2 is April through June, Q3 is July through September, Q4 is October through December.
* A _recession_ is defined as starting with two consecutive quarters of GDP decline, and ending with two consecutive quarters of GDP growth.
* A _recession bottom_ is the quarter within a recession which had the lowest GDP.
* A _university town_ is a city which has a high percentage of university students compared to the total population of the city.

**Hypothesis**: University towns have their mean housing prices less effected by recessions. Run a t-test to compare the ratio of the mean price of houses in university towns the quarter before the recession starts compared to the recession bottom. (`price_ratio=quarter_before_recession/recession_bottom`)

The following data files are available for this assignment:
* From the [Zillow research data site](http://www.zillow.com/research/data/) there is housing data for the United States. In particular the datafile for [all homes at a city level](http://files.zillowstatic.com/research/public/City/City_Zhvi_AllHomes.csv), ```City_Zhvi_AllHomes.csv```, has median home sale prices at a fine grained level.
* From the Wikipedia page on college towns is a list of [university towns in the United States](https://en.wikipedia.org/wiki/List_of_college_towns#College_towns_in_the_United_States) which has been copy and pasted into the file ```university_towns.txt```.
* From Bureau of Economic Analysis, US Department of Commerce, the [GDP over time](http://www.bea.gov/national/index.htm#gdp) of the United States in current dollars (use the chained value in 2009 dollars), in quarterly intervals, in the file ```gdplev.xls```. For this assignment, only look at GDP data from the first quarter of 2000 onward.

Each function in this assignment below is worth 10%, with the exception of ```run_ttest()```, which is worth 50%.

In [419]:
# Use this dictionary to map state names to two letter acronyms
states = {'OH': 'Ohio', 'KY': 'Kentucky', 'AS': 'American Samoa', 'NV': 'Nevada', 'WY': 'Wyoming', 'NA': 'National', 'AL': 'Alabama', 'MD': 'Maryland', 'AK': 'Alaska', 'UT': 'Utah', 'OR': 'Oregon', 'MT': 'Montana', 'IL': 'Illinois', 'TN': 'Tennessee', 'DC': 'District of Columbia', 'VT': 'Vermont', 'ID': 'Idaho', 'AR': 'Arkansas', 'ME': 'Maine', 'WA': 'Washington', 'HI': 'Hawaii', 'WI': 'Wisconsin', 'MI': 'Michigan', 'IN': 'Indiana', 'NJ': 'New Jersey', 'AZ': 'Arizona', 'GU': 'Guam', 'MS': 'Mississippi', 'PR': 'Puerto Rico', 'NC': 'North Carolina', 'TX': 'Texas', 'SD': 'South Dakota', 'MP': 'Northern Mariana Islands', 'IA': 'Iowa', 'MO': 'Missouri', 'CT': 'Connecticut', 'WV': 'West Virginia', 'SC': 'South Carolina', 'LA': 'Louisiana', 'KS': 'Kansas', 'NY': 'New York', 'NE': 'Nebraska', 'OK': 'Oklahoma', 'FL': 'Florida', 'CA': 'California', 'CO': 'Colorado', 'PA': 'Pennsylvania', 'DE': 'Delaware', 'NM': 'New Mexico', 'RI': 'Rhode Island', 'MN': 'Minnesota', 'VI': 'Virgin Islands', 'NH': 'New Hampshire', 'MA': 'Massachusetts', 'GA': 'Georgia', 'ND': 'North Dakota', 'VA': 'Virginia'}

In [420]:
GDP_COL = 'GDP in billions of chained 2009 dollars.1'

In [421]:
def get_list_of_university_towns():
    '''Returns a DataFrame of towns and the states they are in from the 
    university_towns.txt list. The format of the DataFrame should be:
    DataFrame( [ ["Michigan", "Ann Arbor"], ["Michigan", "Yipsilanti"] ], 
    columns=["State", "RegionName"]  )
    
    The following cleaning needs to be done:

    1. For "State", removing characters from "[" to the end.
    2. For "RegionName", when applicable, removing every character from " (" to the end.
    3. Depending on how you read the data, you may need to remove newline character '\n'. '''
    city_data = pd.read_csv('university_towns.txt', sep="\n", header=None)
    city_data['State'] = city_data[0].where(city_data[0].str[-6:]=='[edit]').str[:-6].fillna(method='ffill')
    city_data = city_data[city_data[0].str[-6:]!='[edit]'].rename(columns={0:'RegionName'}).iloc[:,[-1,0]]
    city_data['RegionName'] = city_data['RegionName'].str.split('(').str[0]
    city_data['RegionName'] = city_data['RegionName'].str.strip()
    #city_data['RegionName'] = city_data['RegionName'].str.replace('\s+$','')
    return city_data.sort_values('State').reset_index(drop=True)
#get_list_of_university_towns()

In [422]:
def get_GDP_year_quarter():
    GDP = pd.DataFrame(pd.read_excel('gdplev.xls',header=0,usecols = list(range(0,7)),skiprows=list(range(5)))).iloc[2:].reset_index(drop=True)
    GDP.rename(columns={'Unnamed: 0':'Year','Unnamed: 4':'Quarter'},inplace=True)
    #GDP_y = GDP.iloc[:,:3].set_index('Year').dropna().loc[2000.0:].reset_index(drop=False)
    #GDP_y['Year'] = GDP_y['Year'].astype(int)
    GDP_q = GDP.iloc[:,4:].set_index('Quarter').loc['2000q1':].reset_index(drop=False)
    
    return GDP_q
#get_GDP_year_quarter()

In [430]:
def get_recession_start(GDP_read=None):
    '''Returns the year and quarter of the recession start time as a 
    string value in a format such as 2005q3'''
    if GDP_read is None:
        GDP_q = get_GDP_year_quarter()
    else:
        GDP_q = GDP_read
    
    for i in range(0, len(GDP_q[GDP_COL])-2):
        init_quarter = GDP_q[GDP_COL][i]
        next_quarter = GDP_q[GDP_COL][i+1]
        last_quarter = GDP_q[GDP_COL][i+2]
        if (init_quarter>next_quarter) & (next_quarter>last_quarter):
            rsq = GDP_q['Quarter'][i+1]
            break
    return rsq
get_recession_start()

'2008q2'

In [431]:
def get_recession_end(GDP_read=None):
    '''Returns the year and quarter of the recession end time as a 
    string value in a format such as 2005q3'''
    if GDP_read is None:
        GDP_q = get_GDP_year_quarter()
    else:
        GDP_q = GDP_read
    rsq = get_recession_start(GDP_q)
    for i in range(GDP_q[GDP_q['Quarter']==rsq].index[0]+2, len(GDP_q[GDP_COL])-2):
        init_quarter = GDP_q[GDP_COL][i]
        next_quarter = GDP_q[GDP_COL][i+1]
        last_quarter = GDP_q[GDP_COL][i+2]
        if (init_quarter<next_quarter) & (next_quarter<last_quarter):
            req = GDP_q['Quarter'][i+2]
            break
    return req
#get_recession_end()

'2009q4'

In [432]:
def get_recession_bottom(GDP_read=None,rsq_read=None,req_read=None):
    '''Returns the year and quarter of the recession bottom time as a 
    string value in a format such as 2005q3'''
    if GDP_read is None:
        GDP_q = get_GDP_year_quarter()
    else:
        GDP_q = GDP_read
    if rsq_read is None:
        rsq = get_recession_start(GDP_q)
    else:
        rsq = rsq_read
    if req_read is None:
        req = get_recession_end(GDP_q)
    else:
        req = req_read
    idx_rsq=GDP_q[GDP_q['Quarter']==rsq].index[0]
    idx_req=GDP_q[GDP_q['Quarter']==req].index[0]
    return GDP_q['Quarter'][GDP_q[GDP_COL][idx_rsq:idx_req].idxmin()]
#get_recession_bottom()

'2009q2'

In [426]:
def convert_housing_data_to_quarters():
    '''Converts the housing data to quarters and returns it as mean 
    values in a dataframe. This dataframe should be a dataframe with
    columns for 2000q1 through 2016q3, and should have a multi-index
    in the shape of ["State","RegionName"].
    
    Note: Quarters are defined in the assignment description, they are
    not arbitrary three month periods.
    
    The resulting dataframe should have 67 columns, and 10,730 rows.
    '''
    housing_data = pd.read_csv('City_Zhvi_AllHomes.csv').set_index(['State','RegionName'])
    housing_data = housing_data.drop(list(housing_data.loc[:,'1996-04':'1999-12'].columns),axis=1).iloc[:,4:]
    housing_data.columns = pd.to_datetime(housing_data.columns)
    housing_data = housing_data.groupby(housing_data.columns.to_period('Q'),axis=1).mean()
    housing_data.columns = housing_data.columns.to_series().astype(str).str.lower()
    
    return housing_data.rename(index=states).sort_index(level=0)
#convert_housing_data_to_quarters()

In [434]:
def run_ttest():
    '''First creates new data showing the decline or growth of housing prices
    between the recession start and the recession bottom. Then runs a ttest
    comparing the university town values to the non-university towns values, 
    return whether the alternative hypothesis (that the two groups are the same)
    is true or not as well as the p-value of the confidence. 
    
    Return the tuple (different, p, better) where different=True if the t-test is
    True at a p<0.01 (we reject the null hypothesis), or different=False if 
    otherwise (we cannot reject the null hypothesis). The variable p should
    be equal to the exact p value returned from scipy.stats.ttest_ind(). The
    value for better should be either "university town" or "non-university town"
    depending on which has a lower mean price ratio (which is equivilent to a
    reduced market loss).'''
    # Get data from files
    GDP_q = get_GDP_year_quarter()
    rsq = get_recession_start(GDP_q)
    rsq_idx = GDP_q[GDP_q['Quarter']==rsq].index[0]
    rbq = get_recession_bottom(GDP_q,rsq)
    rbq_idx = GDP_q[GDP_q['Quarter']==rbq].index[0]
    
    # Filter data during recession
    reces_data = convert_housing_data_to_quarters().iloc[:,rsq_idx-1:rbq_idx+1]
    
    # Get university town infos
    ut_infos = get_list_of_university_towns().copy()
    reces_data_ut = pd.merge(reces_data,ut_infos, how='inner',left_index=True,right_on=('State','RegionName')).set_index(['State','RegionName'])
    reces_data_nut = reces_data[~reces_data.index.isin(reces_data_ut.index)]
    reces_data_ut['Ratio'] = reces_data_ut.iloc[:,0]/reces_data_ut.iloc[:,-1]
    reces_data_nut['Ratio'] = reces_data_nut.iloc[:,0]/reces_data_nut.iloc[:,-1]
    
    #T-test
    t,p = ttest_ind(reces_data_ut['Ratio'].dropna(),reces_data_nut['Ratio'].dropna())
    ut_mean = np.mean(reces_data_ut['Ratio'].dropna())
    nut_mean = np.mean(reces_data_nut['Ratio'].dropna())
    if p < 0.01:
        diff = True
    if ut_mean<=nut_mean:
        better = 'university town'
    else:
        better = 'non-university town'
    return diff,p,better
#run_ttest()

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


(True, 0.0040414787138756698, 'university town')